In [ ]:
from pyrho.pyrho import VolumetricFeaturizer
from pyrho.vizualize import get_plotly_scatter_plot
vf = VolumetricFeaturizer.from_hdf5("../test_files/Si/Si.sc.hdf5")
ss = slice(0, -1, 10)
data = vf.data['total'][ss, ss, ss]
dd = data[..., 5]
av=np.linspace(0,1,data.shape[0],endpoint=False)
bv=np.linspace(0,1,data.shape[1],endpoint=False)
AA, BB = np.meshgrid(av,bv,indexing='ij') #indexing to match the lablled array
xx, yy = np.dot(vf.structure.lattice.matrix.T[:2,:2], [AA.flatten(),BB.flatten()])
xshift, yshift = np.dot(vf.structure.lattice.matrix.T[:2,:2], ((av[1]-av[0])/2.,(bv[1]-bv[0])/2.))
print(shift)
ave=np.linspace(0,1,data.shape[0]+1,endpoint=True)
bve=np.linspace(0,1,data.shape[1]+1,endpoint=True)
AAe, BBe = np.meshgrid(ave,bve,indexing='ij') #indexing to match the lablled array
xxe, yye = np.dot(vf.structure.lattice.matrix.T[:2,:2], [AAe.flatten(),BBe.flatten()])
plt.pcolormesh(xxe.reshape(AAe.shape), yye.reshape(AAe.shape), np.log(dd))
plt.scatter(xx+xshift, yy+yshift, c=np.log(dd.flatten()), edgecolors='black')


In [ ]:
def grid_create_from_coordinates(xcoords, ycoords, xcorners=False, ycorners=False, corners=False, domask=False, doarea=False, ctk=ESMF.TypeKind.R8):
    """
    Create a 2 dimensional Grid using the bounds of the x and y coordiantes.
    :param xcoords: The 1st dimension or 'x' coordinates at cell centers, as a Python list or numpy Array
    :param ycoords: The 2nd dimension or 'y' coordinates at cell centers, as a Python list or numpy Array
    :param xcorners: The 1st dimension or 'x' coordinates at cell corners, as a Python list or numpy Array
    :param ycorners: The 2nd dimension or 'y' coordinates at cell corners, as a Python list or numpy Array
    :param domask: boolean to determine whether to set an arbitrary mask or not
    :param doarea: boolean to determine whether to set an arbitrary area values or not
    :param ctk: the coordinate typekind
    :return: grid
    """
    [x, y] = [0, 1]

    # create a grid given the number of grid cells in each dimension, the center stagger location is allocated, the
    # Cartesian coordinate system and type of the coordinates are specified
    max_index = np.array([len(xcoords), len(ycoords)])
    grid = ESMF.Grid(max_index, staggerloc=[ESMF.StaggerLoc.CENTER], coord_sys=ESMF.CoordSys.CART, coord_typekind=ctk)

    # set the grid coordinates using numpy arrays, parallel case is handled using grid bounds
    gridXCenter = grid.get_coords(x)
    x_par = xcoords[grid.lower_bounds[ESMF.StaggerLoc.CENTER][x]:grid.upper_bounds[ESMF.StaggerLoc.CENTER][x]]
    gridXCenter[...] = x_par.reshape((x_par.size, 1))

    gridYCenter = grid.get_coords(y)
    y_par = ycoords[grid.lower_bounds[ESMF.StaggerLoc.CENTER][y]:grid.upper_bounds[ESMF.StaggerLoc.CENTER][y]]
    gridYCenter[...] = y_par.reshape((1, y_par.size))

    # create grid corners in a slightly different manner to account for the bounds format common in CF-like files
    if corners:
        grid.add_coords([ESMF.StaggerLoc.CORNER])
        lbx = grid.lower_bounds[ESMF.StaggerLoc.CORNER][x]
        ubx = grid.upper_bounds[ESMF.StaggerLoc.CORNER][x]
        lby = grid.lower_bounds[ESMF.StaggerLoc.CORNER][y]
        uby = grid.upper_bounds[ESMF.StaggerLoc.CORNER][y]

        gridXCorner = grid.get_coords(x, staggerloc=ESMF.StaggerLoc.CORNER)
        for i0 in range(ubx - lbx - 1):
            gridXCorner[i0, :] = xcorners[i0+lbx, 0]
        gridXCorner[i0 + 1, :] = xcorners[i0+lbx, 1]

        gridYCorner = grid.get_coords(y, staggerloc=ESMF.StaggerLoc.CORNER)
        for i1 in range(uby - lby - 1):
            gridYCorner[:, i1] = ycorners[i1+lby, 0]
        gridYCorner[:, i1 + 1] = ycorners[i1+lby, 1]

    # add an arbitrary mask
    if domask:
        mask = grid.add_item(ESMF.GridItem.MASK)
        mask[:] = 1
        mask[np.where((1.75 <= gridXCenter.any() < 2.25) &
                      (1.75 <= gridYCenter.any() < 2.25))] = 0

    # add arbitrary areas values
    if doarea:
        area = grid.add_item(ESMF.GridItem.AREA)
        area[:] = 5.0

    return grid

def run_regridding(srcfield, dstfield, srcfracfield, dstfracfield):
    # This is for documentation. Do not modify.
    '''
    PRECONDITIONS: Two Fields have been created and a regridding
                   operation is desired from 'srcfield' to 'dstfield'.
                   The 'srcfracfield' and 'dstfractfield' are Fields
                   created to hold the fractions of the source and
                   destination fields which contribute to conservative
                   regridding.\n
    POSTCONDITIONS: A regridding operation has set the data on
                    'dstfield', 'srcfracfield', and 'dstfracfield'.\n
    RETURN VALUES: \n Field :: dstfield \n
                      Field :: srcfracfield \n
                      Field :: dstfracfield \n
    '''
    # call the regridding functions
    regridSrc2Dst = ESMF.Regrid(srcfield, dstfield,
                                regrid_method=ESMF.RegridMethod.CONSERVE,
                                unmapped_action=ESMF.UnmappedAction.ERROR,
                                src_frac_field=srcfracfield,
                                dst_frac_field=dstfracfield)
    dstfield = regridSrc2Dst(srcfield, dstfield)

    return dstfield, srcfracfield, dstfracfield


In [ ]:
gx = xxe.reshape(AAe.shape)
gy = yye.reshape(AAe.shape)
newx, newy = np.meshgrid(np.linspace(0,9,20), np.linspace(0,9,20),indexing='ij') #indexing to match the lablled array

grid1 = grid_create_from_coordinates(xxe,yye)
grid2 = grid_create_from_coordinates(newx.flatten(),newy.flatten())
ESMF.Regrid(grid1, grid2,regrid_method=ESMF.RegridMethod.CONSERVE,
    unmapped_action=ESMF.UnmappedAction.ERROR)


In [ ]:
gx = xxe.reshape(AAe.shape)
gy = yye.reshape(AAe.shape)
grid = ESMF.Grid(np.array(gx.shape), coord_sys=ESMF.CoordSys.CART, num_peri_dims=1,staggerloc=ESMF.StaggerLoc.CENTER)
xpointers = grid.get_coords(coord_dim=0, staggerloc=ESMF.StaggerLoc.CENTER)
ypointers = grid.get_coords(coord_dim=1, staggerloc=ESMF.StaggerLoc.CENTER)

xpointers[...] = gx
ypointers[...] = gy


In [ ]:
newx, newy = np.meshgrid(np.linspace(0,9,20), np.linspace(0,9,20),indexing='ij') #indexing to match the lablled array

grid2 = ESMF.Grid(np.array(newx.shape), coord_sys=ESMF.CoordSys.CART, num_peri_dims=1,staggerloc=ESMF.StaggerLoc.CENTER)
xpointers2 = grid2.get_coords(coord_dim=0, staggerloc=ESMF.StaggerLoc.CENTER)
ypointers2 = grid2.get_coords(coord_dim=1, staggerloc=ESMF.StaggerLoc.CENTER)

xpointers2[...] = newx
ypointers2[...] = newy

regrid = ESMF.Regrid(grid, grid2, 
                     regrid_method=ESMF.RegridMethod.BILINEAR,
                     unmapped_action=ESMF.UnmappedAction.IGNORE,
                     line_type=ESMF.LineType.GREAT_CIRCLE)

In [ ]:
grid.get_coords()

In [ ]:
def grid_create_from_coordinates_periodic(longitudes, latitudes, lon_corners=False, lat_corners=False, corners=False, domask=False):
    """
    Create a 2 dimensional periodic Grid using the 'longitudes' and 'latitudes'.
    :param longitudes: longitude coordinate values at cell centers
    :param latitudes: latitude coordinate values at cell centers
    :param lon_corners: longitude coordinate values at cell corners
    :param lat_corners: latitude coordinate values at cell corners
    :param corners: boolean to determine whether or not to add corner coordinates to this grid
    :param domask: boolean to determine whether to set an arbitrary mask or not
    :return: grid
    """
    [lon, lat] = [0, 1]

    # create a grid given the number of grid cells in each dimension the center stagger location is allocated
    max_index = np.array([len(longitudes), len(latitudes)])
    grid = ESMF.Grid(max_index, num_peri_dims=1, staggerloc=[ESMF.StaggerLoc.CENTER])

    # set the grid coordinates using numpy arrays, parallel case is handled using grid bounds
    gridXCenter = grid.get_coords(lon)
    lon_par = longitudes[grid.lower_bounds[ESMF.StaggerLoc.CENTER][lon]:grid.upper_bounds[ESMF.StaggerLoc.CENTER][lon]]
    gridXCenter[...] = lon_par.reshape((lon_par.size, 1))

    gridYCenter = grid.get_coords(lat)
    lat_par = latitudes[grid.lower_bounds[ESMF.StaggerLoc.CENTER][lat]:grid.upper_bounds[ESMF.StaggerLoc.CENTER][lat]]
    gridYCenter[...] = lat_par.reshape((1, lat_par.size))

    # create grid corners in a slightly different manner to account for the bounds format common in CF-like files
    if corners:
        grid.add_coords([ESMF.StaggerLoc.CORNER])
        lbx = grid.lower_bounds[ESMF.StaggerLoc.CORNER][lon]
        ubx = grid.upper_bounds[ESMF.StaggerLoc.CORNER][lon]
        lby = grid.lower_bounds[ESMF.StaggerLoc.CORNER][lat]
        uby = grid.upper_bounds[ESMF.StaggerLoc.CORNER][lat]

        gridXCorner = grid.get_coords(lon, staggerloc=ESMF.StaggerLoc.CORNER)
        for i0 in range(ubx - lbx - 1):
            gridXCorner[i0, :] = lon_corners[i0+lbx, 0]
        gridXCorner[i0 + 1, :] = lon_corners[i0+lbx, 1]

        gridYCorner = grid.get_coords(lat, staggerloc=ESMF.StaggerLoc.CORNER)
        for i1 in range(uby - lby - 1):
            gridYCorner[:, i1] = lat_corners[i1+lby, 0]
        gridYCorner[:, i1 + 1] = lat_corners[i1+lby, 1]

    # add an arbitrary mask
    if domask:
        mask = grid.add_item(ESMF.GridItem.MASK)
        mask[:] = 1
        mask[np.where((1.75 <= gridXCenter.any() < 2.25) &
                      (1.75 <= gridYCenter.any() < 2.25))] = 0

    return grid

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ESMF

# cell centers
lon_in, lat_in = np.meshgrid(np.linspace(-20, 20, 5), np.linspace(-15, 15, 4))
# cell bounds
lon_in_b, lat_in_b = np.meshgrid(np.linspace(-25, 25, 6), np.linspace(-20, 20, 5))

# stretch the grid in diagonal direction?
# Switch to False will remove the bug
stretch = True
if stretch:
    lon_in += lat_in  
    lat_in += lon_in
    lon_in_b += lat_in_b
    lat_in_b += lon_in_b
    
data = np.arange(20).reshape(4, 5) # a boring data

plt.pcolormesh(lon_in_b, lat_in_b, data)
plt.scatter(lon_in, lat_in, label='cell center')
plt.legend()
plt.title('input data and grid')

In [ ]:
data.shape

In [ ]:
lon_out_b = np.linspace(-50, 50, 51)
lon_out = 0.5*(lon_out_b[1:]+lon_out_b[:-1])

lat_out_b = np.linspace(-80, 80, 51)
lat_out = 0.5*(lat_out_b[1:]+lat_out_b[:-1])

# 1D -> 2D
lon_out, lat_out = np.meshgrid(lon_out, lat_out)
lon_out_b, lat_out_b = np.meshgrid(lon_out_b, lat_out_b)

In [ ]:
# make ESMF.Grid object

grid_in = esmf_grid(lon_in, lat_in)
add_corner(grid_in, lon_in_b, lat_in_b)

grid_out = esmf_grid(lon_out, lat_out)
add_corner(grid_out, lon_out_b, lat_out_b)

In [ ]:
# make ESMF.Regrid object

sourcefield = ESMF.Field(grid_in)
destfield = ESMF.Field(grid_out)
regrid = ESMF.Regrid(sourcefield, destfield, 
                     regrid_method=ESMF.RegridMethod.BILINEAR,
                     unmapped_action=ESMF.UnmappedAction.IGNORE,
                     line_type=ESMF.LineType.GREAT_CIRCLE)

In [ ]:
# Apply regridding

sourcefield.data[...] = data
destfield = regrid(sourcefield, destfield)
data_out = destfield.data

In [ ]:
plt.pcolormesh(lon_out_b, lat_out_b, data_out)

In [ ]:
# make ESMF.Regrid object
regrid_con = ESMF.Regrid(sourcefield, destfield, 
                        regrid_method=ESMF.RegridMethod.CONSERVE,
                        unmapped_action=ESMF.UnmappedAction.IGNORE,
                        )

In [ ]:
# Apply regridding
sourcefield.data[...] = data
destfield = regrid_con(sourcefield, destfield)
data_out_con = destfield.data.copy() # to avoid being overwritten by the next operation

In [ ]:
plt.pcolormesh(lon_out_b, lat_out_b, data_out_con)